In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat


In [33]:
path = 'ex3data1.mat'
data = loadmat(path)
data['X'], data['y']

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8))

In [34]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [35]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [36]:
def cost(theta, X, y, alpha):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    #multiply和*千万不能混为一谈！！！ *为矩阵相乘， multiply只是对应位置元素相乘，*会导致矩阵形状改变
    #遇到的最大的坑是opt.minimize报错ValueError: tnc: invalid gradient vector from minimized function.通过reshape解决
    reg = (alpha / (2 * len(X))) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2))

    return (1 / len(X)) * np.sum(first - second) + reg

如果我们要使用梯度下降法令这个代价函数最小化，因为我们未对${{\theta }_{0}}$ 进行正则化，所以梯度下降算法将分两种情形：
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{{\theta }_{0}}:={{\theta }_{0}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{_{0}}^{(i)}} \\ 
 & \text{     }{{\theta }_{j}}:={{\theta }_{j}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{{\theta }_{j}} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}

以下是原始代码是使用for循环的梯度函数：

In [37]:
def gradient(theta, X, y, alpha):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    error = sigmoid(X * theta.T) - y

    grad = (1 / len(X)) * (error.T * X) + (alpha / len(X)) * theta
    grad[0, 0] = np.sum(np.multiply(error, X[:, 0])) / (len(X))
    #这里又是*和multiply的区别
    return np.array(grad).ravel()

In [38]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, alpha):
    y = np.matrix(y)
    
    rows = X.shape[0]
    params = X.shape[1] + 1
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.matrix(X)

    theta_all = np.zeros((num_labels, params))

    for i in range(1, num_labels + 1):
        theta = np.zeros(params)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        fmin = minimize(fun = cost, x0 = theta, args = (X, y_i, alpha), method = 'TNC', jac = gradient)
        
        print(fmin.x)
        theta_all[i-1, :] = fmin.x
        
    
    return theta_all


In [39]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)
theta = np.zeros(params + 1)

y_0 = np.array([1 if label ==0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, params, all_theta.shape


((5000, 401), (5000, 1), (401,), 400, (10, 401))

In [40]:
all_theta = one_vs_all(data['X'], data['y'], 10, 0.3)
all_theta

6e-01 -9.30777191e-03  1.84049715e-02
  1.88490480e-02  1.36972700e-02  1.10943298e-02  1.40801306e-02
  3.72011792e-03  0.00000000e+00 -6.55683292e-07  3.50267778e-06
  2.01714761e-03 -2.89814888e-02 -6.80211974e-02 -1.40401141e-01
 -4.25585064e-02 -1.28179205e-02  7.80223548e-03 -6.91507167e-02
 -1.53694159e-02  1.75219179e-03 -3.16836790e-02 -5.76566558e-03
 -3.90751823e-03 -8.26686493e-03 -1.86131079e-03  2.24329280e-04
  0.00000000e+00]
[-8.56388168e+00  0.00000000e+00  0.00000000e+00 -1.57540445e-05
  1.42320101e-04  2.22227744e-04  2.27092431e-03 -2.63048253e-02
 -1.88471076e-02  3.39954552e-04 -7.48250015e-04  9.11887208e-05
  2.67732247e-06  7.62236737e-05  6.29702588e-04  1.56124324e-03
  1.01024098e-03 -3.86208301e-05 -1.83761230e-06 -3.61626735e-08
  0.00000000e+00 -3.09409457e-10 -1.86880417e-05  3.34438448e-04
 -1.01814159e-03 -2.78956977e-03  2.88540210e-02 -6.83950091e-03
 -7.04761604e-02 -1.07252144e-01 -4.47420295e-02  2.23735092e-02
  4.93452917e-03 -5.91625852e-03  

array([[-2.77767481e+00,  0.00000000e+00,  0.00000000e+00, ...,
         3.64877507e-03, -3.34478989e-11,  0.00000000e+00],
       [-3.41333265e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.04163313e-02, -1.18084226e-03,  0.00000000e+00],
       [-5.21896054e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -4.81235738e-05, -3.46738657e-07,  0.00000000e+00],
       ...,
       [-8.56388168e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.28822417e-04,  2.36320285e-05,  0.00000000e+00],
       [-5.06692038e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -5.03053203e-03,  3.85313579e-04,  0.00000000e+00],
       [-6.47314185e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.69673382e-05,  4.33915979e-06,  0.00000000e+00]])

In [41]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

In [42]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))
#这个函数的意思就是将function应用于iterable的每一个元素，结果以列表的形式返回。
#map函数的原型是map(function, iterable, …)，它的返回结果是一个列表。

accuracy = 95.5%
